In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import cufflinks as cf


In [7]:
items = pd.read_csv("../input/items.csv")
shops = pd.read_csv("../input/shops.csv")
item_cat = pd.read_csv("../input/item_categories.csv")
sales_train = pd.read_csv("../input/sales_train.csv")
test= pd.read_csv("../input/test.csv")

In [8]:
test.drop("ID", axis = 1, inplace = True)

In [9]:
sales_train.shape

In [10]:
#since we have no idea what the month of the test block is we assume its the next month of the data series

test["date_block_num"] = 34

In [11]:
#create pivot table for the average item price based on item IDs 

itemidpivot = sales_train.pivot_table( values = "item_price", index = "item_id",)

In [12]:
test1 = test.join(itemidpivot, on = 'item_id')

In [13]:
sales_train1 = sales_train.join(items.set_index("item_id"), on = "item_id")

In [14]:
#create pivot table for the average item price based on item categories (for still missing prices)

itemcatpivot = sales_train1.pivot_table(values = "item_price", index = "item_category_id")

In [15]:
test2 = test1.join(items.set_index("item_id"), on = "item_id")

In [16]:
test2

In [17]:
test2.info()


In [18]:
itemcatpivot.reset_index(inplace = True)

In [19]:
itemcatpivot.rename(columns={'item_price': 'cat_price'}, inplace = True)

In [20]:
itemcatpivot["id"]= itemcatpivot["item_category_id"].astype("int64", copy = False)

In [21]:
itemcatpivot.drop(["item_category_id"] ,axis = 1,  inplace = True)

In [22]:
test3 = test2.join(itemcatpivot, on = "item_category_id")

In [23]:
test3["item_price"][test3["item_price"].isnull()] = test3["cat_price"]

In [24]:
test3.isnull().sum()

In [25]:
test3.drop(["id" ,"cat_price"] , axis = 1, inplace = True)

In [26]:
sales_train1.drop(["date"], axis = 1, inplace= True)

In [27]:
sales_train1.head(5)

In [28]:
test3.head(5)

In [29]:
totaldf = sales_train1.append(test3)

In [30]:
sales_train2 = sales_train1.drop(["item_name"], axis = 1 )

In [31]:
sales_train2.head(5)

In [32]:
test4 = test3.drop(["item_name"], axis = 1)

In [33]:
test4.head(5)

In [34]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [35]:
totaldf = sales_train2.append(test4)

In [36]:
totaldf.head(5)

In [37]:
ntrain = 2935849
train = totaldf[:ntrain]
test = totaldf[ntrain:]

In [38]:
dtree = DecisionTreeClassifier()
rforest = RandomForestClassifier()
LinearModel = LinearRegression()


In [39]:
X_train = train.drop(["item_cnt_day"], axis = 1)
y_train = train["item_cnt_day"]
X_test = test.drop(["item_cnt_day"], axis = 1)

dtree.fit(X_train, y_train)
dtreepredict = dtree.predict(X_test)

In [40]:
rforest.fit(X_train, y_train)
rforestpredict = rforest.predict(X_test)

In [41]:
LinearModel.fit(X_train, y_train)
LinearModelpredict = LinearModel.predict(X_test)

In [50]:
sub = pd.DataFrame()
sub['ID'] = test.index
sub['item_cnt_month'] = rforestpredict 
sub.to_csv('output.csv',index=False)

In [51]:
sample = pd.read_csv("../input/sample_submission.csv")

In [52]:
sample

In [53]:
sub